In [ ]:

import os
#import test_strategy as strategy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
#import dateparse
import tailer
import io

In [ ]:
def strategy(close, n1=20, n2=5):
    global mean1, mean2, num1, initial, close_series
    if(n1>n2 and n2>0):
        if(num1<n1):
            close_series.append(close)
            num1+=1
        else:
            close_series.pop(0)
            close_series.append(close)
            mean1 = sum(close_series)/n1
            mean2 = sum(close_series[-n2:])/n2

        if(initial==0 and mean1<mean2):
            initial=1
            return "buy"
        elif(initial==1 and mean1>mean2):
            initial=0
            return "sell"

In [ ]:
def main(ticker_symbol,no_of_ticks = 20,max_investment = 10000, brokerage = 0, allow_short = False, n=20, m=5):

    # max_investment = 5000
    #brokerage = 30
    #allow_short = False
    #ticker_symbol = "ADANIENT.NS"
    path = '/home/mgoel/Documents/quant_algos/Broker_simulation/Data/'+ticker_symbol+'.csv'
    #no_of_ticks = 20

    if(os.path.exists(path.strip())==False):
        print("Data file does not exist")
        print(path)
        exit()

    with open(path) as file:
        last_lines = tailer.tail(file, no_of_ticks)

    dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
    data = pd.read_csv(io.StringIO('\n'.join(last_lines)), header=None, index_col=0, parse_dates=[0], date_parser=dateparse)

    #buy order
    def buy_order(price):
        global no_of_buys
        global total_buy
        if(total_buy-total_sells>max_investment):
            return
        no_of_buys+=1
        total_buy+=price

    
    #buy order
    def sell_order(price):
        global no_of_sells, total_sells
        if(total_sells-total_buy>max_investment):
            return
        if(allow_short==False and no_of_sells>=no_of_buys):
            return
        no_of_sells+=1
        total_sells+=price - brokerage


    global pnl
    global net_profit
    for n in range(no_of_ticks):
        close = data.iloc[n][4]
        strategy_return = strategy(close, n1=n, n2=m)
        if(strategy_return=="buy"):
            buy_order(close)
        elif(strategy_return=="sell"):
            sell_order(close)
        pnl.append(total_sells-total_buy)
        net_profit.append(total_sells-total_buy + ((no_of_buys - no_of_sells )*close) )

    # y_axis_1 = pd.Series(pnl, index = data.index)
    # y_axis_2 = pd.Series(net_profit, index = data.index)
    # plt.plot(y_axis_1,color='blue', label="Booked profits")
    # plt.plot(y_axis_2, color='green', label = "Net profits")
    # # print("Current_holdings = ",no_of_buys - no_of_sells )
    # # print("Net profit = ", net_profit[-1])
    # # print("Number of buys = ", no_of_buys)
    # # print("Number of sells = ", no_of_sells)
    # # print("Profit by holding ", no_of_buys-no_of_sells, "shares = ", (no_of_buys-no_of_sells)*(data.iloc[-1][4] - data.iloc[0][4]))
    # # plt.legend(loc='upper left', fontsize=8)
    # # plt.show()
    return net_profit[-1]

In [ ]:
n = 40
m = 10
max_profit = -5000
n_opt = n
m_opt = m
for n1 in range(n):
    for n2 in range(m):
        if(n1<=n2 or n1==0 or n2==0):
            continue

        mean1 = 0
        mean2 = 0
        num1 = 0
        initial = 1 #sum1>sum2
        close_series = []

        #buy variables
        no_of_buys = 0
        total_buy = 0

        #sell variables
        no_of_sells = 0
        total_sells = 0

        strategy_return = "none"
        pnl = []
        net_profit = []

        profit = main('HDFCBANK.NS',no_of_ticks=400,n=n1,m=n2)
        if(profit>max_profit):
            max_profit=profit
            n_opt = n1
            m_opt = n2
print("Max profit = ", max_profit)
print("Optimized n = ", n_opt)
print("Optimized m = ", m_opt)
